請就所提供的資料以財務特徵(X1-X95)預測公司是否破產(Y)，並新增衍生變數，比較不同的機器學習模型，並說明分析結果

(Precision, Recall, Accuracy若未達0.9，請說明如何發展衍生變數使模式具有良好的預測能力)。

類別:
Net Income Flag(94)、Liability-Assets Flag(85)、Bankrupt?

#載入檔案和套件

In [146]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from scipy import stats
from sklearn import metrics

In [147]:
data25=pd.read_excel("/content/drive/MyDrive/機器學習/闖關/預測公司破產資料.xlsx")

In [148]:
f=data25.groupby('Bankrupt?')
frequence=f.size()
flist=pd.DataFrame(frequence,columns=['Frequence_是否破產'])
f=data25.groupby('Bankrupt?')
percentage=f.size()/len(data25)
plist=pd.DataFrame(percentage,columns=['Percentage_是否破產'])
a=flist.merge(plist,on = 'Bankrupt?')
a

,Frequence_是否破產,Percentage_是否破產
Bankrupt?,,
0.000,6599,0.968
1.000,220,0.032


#***訓練集:測試集=75:25***

#原始資料 混淆矩陣
#當Bankrupt?=1時Precision, Recall的值小於0.9所以須新增衍生變數使其變好看

In [291]:
#給下面自變數X取除了Bankrupt?以外的欄位名稱，不然一個一個打會打到亂掉
a25=data25.copy()
a25=a25.drop('Bankrupt?',axis=1)

In [292]:
X=a25[a25.columns]
y=data25['Bankrupt?']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

決策樹

In [302]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [303]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4937    0]
 [  24  153]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4937
         1.0       1.00      0.86      0.93       177

    accuracy                           1.00      5114
   macro avg       1.00      0.93      0.96      5114
weighted avg       1.00      1.00      1.00      5114



In [304]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[1621   41]
 [  32   11]]

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      1662
         1.0       0.21      0.26      0.23        43

    accuracy                           0.96      1705
   macro avg       0.60      0.62      0.60      1705
weighted avg       0.96      0.96      0.96      1705



隨機森林

In [154]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [155]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4937    0]
 [   9  168]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4937
         1.0       1.00      0.95      0.97       177

    accuracy                           1.00      5114
   macro avg       1.00      0.97      0.99      5114
weighted avg       1.00      1.00      1.00      5114



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [156]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[1642   20]
 [  31   12]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1662
         1.0       0.38      0.28      0.32        43

    accuracy                           0.97      1705
   macro avg       0.68      0.63      0.65      1705
weighted avg       0.97      0.97      0.97      1705



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


羅吉斯回歸

In [157]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [158]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4908   29]
 [ 172    5]]

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      4937
         1.0       0.15      0.03      0.05       177

    accuracy                           0.96      5114
   macro avg       0.56      0.51      0.51      5114
weighted avg       0.94      0.96      0.95      5114



In [159]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[1658    4]
 [  43    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      1662
         1.0       0.00      0.00      0.00        43

    accuracy                           0.97      1705
   macro avg       0.49      0.50      0.49      1705
weighted avg       0.95      0.97      0.96      1705



KNN

In [160]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [161]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4934    3]
 [ 157   20]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       0.87      0.11      0.20       177

    accuracy                           0.97      5114
   macro avg       0.92      0.56      0.59      5114
weighted avg       0.97      0.97      0.96      5114



In [162]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[1652   10]
 [  42    1]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1662
         1.0       0.09      0.02      0.04        43

    accuracy                           0.97      1705
   macro avg       0.53      0.51      0.51      1705
weighted avg       0.95      0.97      0.96      1705



SVM(因資料沒處理過所以會跑很慢，故不採用)

In [163]:
# from sklearn import svm 
# clf = svm.SVC(kernel = 'linear') 
# clf.fit(X_train,y_train) 

In [164]:
# #訓練
# from sklearn.metrics import classification_report,confusion_matrix
# clf_tpred= clf.predict(X_train)
# print(confusion_matrix(y_train,clf_tpred))
# print("======================================================\n")
# print(classification_report(y_train,clf_tpred))

In [165]:
# #測試
# clf_pred= clf.predict(X_test)
# print(confusion_matrix(y_test,clf_pred))
# print("======================================================\n")
# print(classification_report(y_test,clf_pred))

#標準化&卡方檢定t檢定

In [166]:
#最小最大值標準化
from sklearn import preprocessing
minmax=preprocessing.MinMaxScaler()
dataMinMax=minmax.fit_transform(data25)
dataMinMax=pd.DataFrame(dataMinMax,columns=['Bankrupt?', ' ROA(C) before interest and depreciation before interest',' ROA(A) before interest and % after tax',
       ' ROA(B) before interest and depreciation after tax',' Operating Gross Margin', ' Realized Sales Gross Margin',' Operating Profit Rate', 
       ' Pre-tax net Interest Rate',' After-tax net Interest Rate',' Non-industry income and expenditure/revenue',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',' Research and development expense rate', ' Cash flow rate',
       ' Interest-bearing debt interest rate', ' Tax rate (A)',' Net Value Per Share (B)', ' Net Value Per Share (A)',
       ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons',' Cash Flow Per Share', ' Revenue Per Share (Yuan 瞼)',
       ' Operating Profit Per Share (Yuan 瞼)',' Per Share Net profit before tax (Yuan 瞼)',' Realized Sales Gross Profit Growth Rate',
       ' Operating Profit Growth Rate', ' After-tax Net Profit Growth Rate',' Regular Net Profit Growth Rate', ' Continuous Net Profit Growth Rate',
       ' Total Asset Growth Rate', ' Net Value Growth Rate',' Total Asset Return Growth Rate Ratio', ' Cash Reinvestment %',
       ' Current Ratio', ' Quick Ratio', ' Interest Expense Ratio',' Total debt/Total net worth', ' Debt ratio %', ' Net worth/Assets',
       ' Long-term fund suitability ratio (A)', ' Borrowing dependency',' Contingent liabilities/Net worth',' Operating profit/Paid-in capital',
       ' Net profit before tax/Paid-in capital',' Inventory and accounts receivable/Net value', ' Total Asset Turnover',
       ' Accounts Receivable Turnover', ' Average Collection Days',' Inventory Turnover Rate (times)', ' Fixed Assets Turnover Frequency',
       ' Net Worth Turnover Rate (times)', ' Revenue per person',' Operating profit per person', ' Allocation rate per person',
       ' Working Capital to Total Assets', ' Quick Assets/Total Assets',' Current Assets/Total Assets', ' Cash/Total Assets',
       ' Quick Assets/Current Liability', ' Cash/Current Liability',' Current Liability to Assets', ' Operating Funds to Liability',
       ' Inventory/Working Capital', ' Inventory/Current Liability',' Current Liabilities/Liability', ' Working Capital/Equity',
       ' Current Liabilities/Equity', ' Long-term Liability to Current Assets',' Retained Earnings to Total Assets', ' Total income/Total expense',
       ' Total expense/Assets', ' Current Asset Turnover Rate',' Quick Asset Turnover Rate', ' Working capitcal Turnover Rate',
       ' Cash Turnover Rate', ' Cash Flow to Sales', ' Fixed Assets to Assets',' Current Liability to Liability', ' Current Liability to Equity',
       ' Equity to Long-term Liability', ' Cash Flow to Total Assets',' Cash Flow to Liability', ' CFO to Assets', ' Cash Flow to Equity',
       ' Current Liability to Current Assets', ' Liability-Assets Flag',' Net Income to Total Assets', ' Total assets to GNP price',
       ' No-credit Interval', ' Gross Profit to Sales'," Net Income to Stockholder's Equity", ' Liability to Equity',
       ' Degree of Financial Leverage (DFL)',' Interest Coverage Ratio (Interest expense to EBIT)',' Net Income Flag', ' Equity to Liability'])
dataMinMax.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1.000,0.371,0.424,0.406,0.601,0.601,0.999,0.797,0.809,0.303,...,0.717,0.000,0.623,0.601,0.828,0.290,0.027,0.564,0.000,0.016
1,1.000,0.464,0.538,0.517,0.610,0.610,0.999,0.797,0.809,0.304,...,0.795,0.000,0.624,0.610,0.840,0.284,0.265,0.570,0.000,0.021
2,1.000,0.426,0.499,0.472,0.601,0.601,0.999,0.796,0.808,0.302,...,0.775,0.000,0.624,0.601,0.837,0.290,0.027,0.564,0.000,0.016
3,1.000,0.400,0.451,0.458,0.584,0.584,0.999,0.797,0.809,0.303,...,0.740,0.000,0.623,0.584,0.835,0.282,0.027,0.565,0.000,0.024
4,1.000,0.465,0.538,0.522,0.599,0.599,0.999,0.797,0.809,0.303,...,0.795,0.000,0.624,0.599,0.840,0.279,0.025,0.576,0.000,0.035


In [167]:
#1類對多量的t檢定

data_set2=[' ROA(C) before interest and depreciation before interest',' ROA(A) before interest and % after tax',' ROA(B) before interest and depreciation after tax',
       ' Operating Gross Margin', ' Realized Sales Gross Margin',' Operating Profit Rate', ' Pre-tax net Interest Rate',' After-tax net Interest Rate',
       ' Non-industry income and expenditure/revenue',' Continuous interest rate (after tax)', ' Operating Expense Rate',' Research and development expense rate',
       ' Cash flow rate',' Interest-bearing debt interest rate', ' Tax rate (A)',' Net Value Per Share (B)', ' Net Value Per Share (A)',
       ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons',' Cash Flow Per Share', ' Revenue Per Share (Yuan 瞼)',
       ' Operating Profit Per Share (Yuan 瞼)',' Per Share Net profit before tax (Yuan 瞼)',' Realized Sales Gross Profit Growth Rate',
       ' Operating Profit Growth Rate', ' After-tax Net Profit Growth Rate',' Regular Net Profit Growth Rate', ' Continuous Net Profit Growth Rate',
       ' Total Asset Growth Rate', ' Net Value Growth Rate',' Total Asset Return Growth Rate Ratio', ' Cash Reinvestment %',
       ' Current Ratio', ' Quick Ratio', ' Interest Expense Ratio',' Total debt/Total net worth', ' Debt ratio %', ' Net worth/Assets',
       ' Long-term fund suitability ratio (A)', ' Borrowing dependency',' Contingent liabilities/Net worth',' Operating profit/Paid-in capital',
       ' Net profit before tax/Paid-in capital',' Inventory and accounts receivable/Net value', ' Total Asset Turnover',' Accounts Receivable Turnover',
       ' Average Collection Days',' Inventory Turnover Rate (times)', ' Fixed Assets Turnover Frequency',' Net Worth Turnover Rate (times)',
       ' Revenue per person',' Operating profit per person', ' Allocation rate per person',' Working Capital to Total Assets',
       ' Quick Assets/Total Assets',' Current Assets/Total Assets', ' Cash/Total Assets',' Quick Assets/Current Liability', ' Cash/Current Liability',
       ' Current Liability to Assets', ' Operating Funds to Liability',' Inventory/Working Capital', ' Inventory/Current Liability',
       ' Current Liabilities/Liability', ' Working Capital/Equity',' Current Liabilities/Equity', ' Long-term Liability to Current Assets',
       ' Retained Earnings to Total Assets', ' Total income/Total expense',' Total expense/Assets', ' Current Asset Turnover Rate',
       ' Quick Asset Turnover Rate', ' Working capitcal Turnover Rate',' Cash Turnover Rate', ' Cash Flow to Sales', ' Fixed Assets to Assets',
       ' Current Liability to Liability', ' Current Liability to Equity',' Equity to Long-term Liability', ' Cash Flow to Total Assets',
       ' Cash Flow to Liability', ' CFO to Assets', ' Cash Flow to Equity',' Current Liability to Current Assets',' Net Income to Total Assets',
       ' Total assets to GNP price',' No-credit Interval', ' Gross Profit to Sales'," Net Income to Stockholder's Equity",
       ' Liability to Equity',' Degree of Financial Leverage (DFL)',' Interest Coverage Ratio (Interest expense to EBIT)',' Equity to Liability']
name=[]
alist=[]
pvlist=[]
for i in range(len(data_set2)):
  (a,pv)=stats.ttest_ind(dataMinMax[data_set2[i]][dataMinMax['Bankrupt?']==0],dataMinMax[data_set2[i]][dataMinMax['Bankrupt?']==1])
  name.append(data_set2[i])
  alist.append(a)
  pvlist.append(pv)
pd.set_option('display.float_format',lambda x:'%.3f'%x)
result=pd.DataFrame(alist,name,columns=['t_value'])
result.insert(1,'p_value',pvlist)
result=result.sort_values(by=['p_value'])[::1]
result

,t_value,p_value
Net Income to Total Assets,27.447,0.000
ROA(A) before interest and % after tax,24.356,0.000
ROA(B) before interest and depreciation after tax,23.435,0.000
ROA(C) before interest and depreciation before interest,22.305,0.000
Net worth/Assets,21.333,0.000
...,...,...
Inventory/Current Liability,-0.068,0.946
Long-term Liability to Current Assets,-0.064,0.949
Cash Flow to Sales,-0.040,0.968
Realized Sales Gross Profit Growth Rate,0.038,0.970


In [168]:
#1類對多類的卡方檢定

qv3=[' Net Income Flag',' Liability-Assets Flag']
alist = []
pvlist = []
name = []
for i in qv3:
  cr=stats.chi2_contingency(pd.crosstab(dataMinMax['Bankrupt?'],dataMinMax[i]))
  pvlist.append(cr[1:2])
  alist.append(cr[0:1])
  name.append(i)
pd.set_option('display.float_format',lambda x:'%.3f'%x)
result=pd.DataFrame(alist,name,columns=['Chi-Square'])
pvlist2 = [item for t in pvlist for item in t] #Tuple to List
result.insert(1,'p_value',pvlist2)
result=result.sort_values(by=['p_value'])[::-1]
result

,Chi-Square,p_value
Net Income Flag,0.000,1.000
Liability-Assets Flag,110.138,0.000


In [169]:
dataP25=dataMinMax[[' Liability-Assets Flag',' ROA(A) before interest and % after tax',' ROA(B) before interest and depreciation after tax',
       ' ROA(C) before interest and depreciation before interest',' Net worth/Assets', ' Debt ratio %',' Persistent EPS in the Last Four Seasons',
       ' Per Share Net profit before tax (Yuan 瞼)', ' Borrowing dependency',' Net Value Per Share (A)', ' Net Value Per Share (B)',
       ' Net Value Per Share (C)', ' Operating Profit Per Share (Yuan 瞼)',' Operating profit/Paid-in capital', ' Tax rate (A)',
       ' Operating Gross Margin', ' Realized Sales Gross Margin',' Cash Flow Per Share', ' Cash flow rate',' Contingent liabilities/Net worth',
       ' Net Value Growth Rate',' Cash Reinvestment %', ' Total Asset Growth Rate',' After-tax Net Profit Growth Rate',
       ' Regular Net Profit Growth Rate',' Quick Ratio', ' Research and development expense rate',' Net Income to Total Assets',
       ' Retained Earnings to Total Assets',' Net profit before tax/Paid-in capital',' Current Liability to Assets', ' Working Capital to Total Assets',
       " Net Income to Stockholder's Equity",' Current Liability to Current Assets', ' Liability to Equity',' Current Liability to Equity',
       ' Current Liabilities/Equity',' Working Capital/Equity', ' Total expense/Assets',' Equity to Long-term Liability', ' CFO to Assets',
       ' Cash/Total Assets',' Gross Profit to Sales',' Operating profit per person', ' Quick Assets/Total Assets',' Equity to Liability',
       ' Cash/Current Liability',' Operating Funds to Liability',' Inventory and accounts receivable/Net value',' Fixed Assets Turnover Frequency',
       ' Cash Flow to Total Assets',' Total Asset Turnover', ' Fixed Assets to Assets',' Cash Flow to Equity', ' Current Assets/Total Assets',
       ' Cash Flow to Liability', ' Revenue per person',' Total assets to GNP price', ' Quick Asset Turnover Rate']]

#標準化&降維後的P值小於0.05的欄位當自變數的混淆矩陣

#PCA降維

In [170]:
#PCA降維
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
principalComponents = pca.fit_transform(dataP25)
dataPPCA25 = pd.DataFrame(data=principalComponents)
dataPPCA25.head()

,0,1,2,3,4,5
0,0.597,-0.005,0.164,-0.172,-0.251,-0.203
1,0.710,-0.002,0.057,-0.158,-0.158,-0.015
2,-0.068,0.258,-0.098,-0.244,0.073,-0.287
3,0.688,0.471,0.313,-0.116,0.503,-0.148
4,0.365,-0.061,0.062,-0.176,-0.153,-0.066


In [171]:
X=dataPPCA25[dataPPCA25.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

決策樹

In [172]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [173]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4930    7]
 [  46  131]]

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      4937
         1.0       0.95      0.74      0.83       177

    accuracy                           0.99      5114
   macro avg       0.97      0.87      0.91      5114
weighted avg       0.99      0.99      0.99      5114



In [174]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[1620   42]
 [  31   12]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      1662
         1.0       0.22      0.28      0.25        43

    accuracy                           0.96      1705
   macro avg       0.60      0.63      0.61      1705
weighted avg       0.96      0.96      0.96      1705



隨機森林

In [175]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [176]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4937    0]
 [  13  164]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4937
         1.0       1.00      0.93      0.96       177

    accuracy                           1.00      5114
   macro avg       1.00      0.96      0.98      5114
weighted avg       1.00      1.00      1.00      5114



In [177]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[1647   15]
 [  34    9]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1662
         1.0       0.38      0.21      0.27        43

    accuracy                           0.97      1705
   macro avg       0.68      0.60      0.63      1705
weighted avg       0.96      0.97      0.97      1705



羅吉斯回歸

In [178]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [179]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4935    2]
 [ 161   16]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       0.89      0.09      0.16       177

    accuracy                           0.97      5114
   macro avg       0.93      0.54      0.57      5114
weighted avg       0.97      0.97      0.96      5114



In [180]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[1654    8]
 [  38    5]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      1662
         1.0       0.38      0.12      0.18        43

    accuracy                           0.97      1705
   macro avg       0.68      0.56      0.58      1705
weighted avg       0.96      0.97      0.97      1705



KNN

In [181]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [182]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4921   16]
 [ 108   69]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4937
         1.0       0.81      0.39      0.53       177

    accuracy                           0.98      5114
   macro avg       0.90      0.69      0.76      5114
weighted avg       0.97      0.98      0.97      5114



In [183]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[1644   18]
 [  39    4]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1662
         1.0       0.18      0.09      0.12        43

    accuracy                           0.97      1705
   macro avg       0.58      0.54      0.55      1705
weighted avg       0.96      0.97      0.96      1705



SVM

In [184]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [185]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4937    0]
 [ 177    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       0.00      0.00      0.00       177

    accuracy                           0.97      5114
   macro avg       0.48      0.50      0.49      5114
weighted avg       0.93      0.97      0.95      5114



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [186]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[1662    0]
 [  43    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      1662
         1.0       0.00      0.00      0.00        43

    accuracy                           0.97      1705
   macro avg       0.49      0.50      0.49      1705
weighted avg       0.95      0.97      0.96      1705



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#LDA降維

In [187]:
#LDA降維
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1) #使用LDA降維，因此資料集的目標變數有2個類別，所以最多至1維
lda_x = lda.fit_transform(dataP25[dataP25.columns],dataMinMax['Bankrupt?'])
dataPLDA25 = pd.DataFrame(data=lda_x)
dataPLDA25.head()

,0
0,3.804
1,0.931
2,4.860
3,1.697
4,-0.061


In [188]:
X=dataPLDA25[dataPLDA25.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

決策樹

In [189]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [190]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4934    3]
 [ 102   75]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4937
         1.0       0.96      0.42      0.59       177

    accuracy                           0.98      5114
   macro avg       0.97      0.71      0.79      5114
weighted avg       0.98      0.98      0.98      5114



In [191]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[1638   24]
 [  32   11]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1662
         1.0       0.31      0.26      0.28        43

    accuracy                           0.97      1705
   macro avg       0.65      0.62      0.63      1705
weighted avg       0.96      0.97      0.97      1705



隨機森林

In [192]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [193]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4935    2]
 [  16  161]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4937
         1.0       0.99      0.91      0.95       177

    accuracy                           1.00      5114
   macro avg       0.99      0.95      0.97      5114
weighted avg       1.00      1.00      1.00      5114



In [194]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[1614   48]
 [  28   15]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      1662
         1.0       0.24      0.35      0.28        43

    accuracy                           0.96      1705
   macro avg       0.61      0.66      0.63      1705
weighted avg       0.96      0.96      0.96      1705



羅吉斯回歸

In [195]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [196]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4907   30]
 [ 134   43]]

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      4937
         1.0       0.59      0.24      0.34       177

    accuracy                           0.97      5114
   macro avg       0.78      0.62      0.66      5114
weighted avg       0.96      0.97      0.96      5114



In [197]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[1650   12]
 [  32   11]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1662
         1.0       0.48      0.26      0.33        43

    accuracy                           0.97      1705
   macro avg       0.73      0.62      0.66      1705
weighted avg       0.97      0.97      0.97      1705



KNN

In [198]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [199]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4916   21]
 [ 102   75]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4937
         1.0       0.78      0.42      0.55       177

    accuracy                           0.98      5114
   macro avg       0.88      0.71      0.77      5114
weighted avg       0.97      0.98      0.97      5114



In [200]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[1632   30]
 [  29   14]]

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      1662
         1.0       0.32      0.33      0.32        43

    accuracy                           0.97      1705
   macro avg       0.65      0.65      0.65      1705
weighted avg       0.97      0.97      0.97      1705



SVM

In [201]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [202]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4937    0]
 [ 165   12]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       1.00      0.07      0.13       177

    accuracy                           0.97      5114
   macro avg       0.98      0.53      0.56      5114
weighted avg       0.97      0.97      0.95      5114



In [203]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[1658    4]
 [  41    2]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      1662
         1.0       0.33      0.05      0.08        43

    accuracy                           0.97      1705
   macro avg       0.65      0.52      0.53      1705
weighted avg       0.96      0.97      0.96      1705



#ICA降維

In [204]:
#ICA降維
from sklearn.decomposition import FastICA
ica = FastICA(n_components=6)
Ica25 = ica.fit_transform(dataP25)
dataPICA25 = pd.DataFrame(data=Ica25)
dataPICA25.head()

,0,1,2,3,4,5
0,-0.005,-0.013,-0.023,-0.008,0.011,-0.010
1,-0.002,-0.016,-0.007,-0.012,0.012,-0.012
2,0.008,0.007,-0.021,-0.004,-0.012,-0.007
3,0.040,-0.000,-0.012,-0.008,0.012,-0.010
4,-0.005,-0.007,-0.009,-0.006,0.007,-0.010


In [205]:
X=dataPICA25[dataPICA25.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25 ,random_state=333)

決策樹

In [206]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [207]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4936    1]
 [  49  128]]

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      4937
         1.0       0.99      0.72      0.84       177

    accuracy                           0.99      5114
   macro avg       0.99      0.86      0.92      5114
weighted avg       0.99      0.99      0.99      5114



In [208]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[1628   34]
 [  36    7]]

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      1662
         1.0       0.17      0.16      0.17        43

    accuracy                           0.96      1705
   macro avg       0.57      0.57      0.57      1705
weighted avg       0.96      0.96      0.96      1705



隨機森林

In [209]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [210]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4936    1]
 [  12  165]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4937
         1.0       0.99      0.93      0.96       177

    accuracy                           1.00      5114
   macro avg       1.00      0.97      0.98      5114
weighted avg       1.00      1.00      1.00      5114



In [211]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[1649   13]
 [  36    7]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1662
         1.0       0.35      0.16      0.22        43

    accuracy                           0.97      1705
   macro avg       0.66      0.58      0.60      1705
weighted avg       0.96      0.97      0.97      1705



羅吉斯回歸

In [212]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [213]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4937    0]
 [ 177    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       0.00      0.00      0.00       177

    accuracy                           0.97      5114
   macro avg       0.48      0.50      0.49      5114
weighted avg       0.93      0.97      0.95      5114



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [214]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[1662    0]
 [  43    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      1662
         1.0       0.00      0.00      0.00        43

    accuracy                           0.97      1705
   macro avg       0.49      0.50      0.49      1705
weighted avg       0.95      0.97      0.96      1705



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN

In [215]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [216]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4921   16]
 [ 101   76]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4937
         1.0       0.83      0.43      0.57       177

    accuracy                           0.98      5114
   macro avg       0.90      0.71      0.78      5114
weighted avg       0.97      0.98      0.97      5114



In [217]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[1643   19]
 [  35    8]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      1662
         1.0       0.30      0.19      0.23        43

    accuracy                           0.97      1705
   macro avg       0.64      0.59      0.61      1705
weighted avg       0.96      0.97      0.96      1705



SVM

In [218]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [219]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4937    0]
 [ 177    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4937
         1.0       0.00      0.00      0.00       177

    accuracy                           0.97      5114
   macro avg       0.48      0.50      0.49      5114
weighted avg       0.93      0.97      0.95      5114



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [220]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[1662    0]
 [  43    0]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      1662
         1.0       0.00      0.00      0.00        43

    accuracy                           0.97      1705
   macro avg       0.49      0.50      0.49      1705
weighted avg       0.95      0.97      0.96      1705



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#***訓練集:測試集=67:33***

#原始資料 混淆矩陣
#當Bankrupt?=1時Precision, Recall的值小於0.9所以須新增衍生變數使其變好看

In [221]:
data33=pd.read_excel("/content/drive/MyDrive/機器學習/闖關/預測公司破產資料.xlsx")

In [222]:
#給下面自變數X取除了Bankrupt?以外的欄位名稱，不然一個一個打會打到亂掉
a33=data33.copy()
a33=a33.drop('Bankrupt?',axis=1)

In [223]:
X=a33[a33.columns]
y=data33['Bankrupt?']
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33 ,random_state=333)

決策樹

In [224]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [225]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4402    0]
 [  23  143]]

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      4402
         1.0       1.00      0.86      0.93       166

    accuracy                           0.99      4568
   macro avg       1.00      0.93      0.96      4568
weighted avg       0.99      0.99      0.99      4568



In [226]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[2142   55]
 [  40   14]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      2197
         1.0       0.20      0.26      0.23        54

    accuracy                           0.96      2251
   macro avg       0.59      0.62      0.60      2251
weighted avg       0.96      0.96      0.96      2251



隨機森林

In [227]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [228]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4402    0]
 [  12  154]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4402
         1.0       1.00      0.93      0.96       166

    accuracy                           1.00      4568
   macro avg       1.00      0.96      0.98      4568
weighted avg       1.00      1.00      1.00      4568



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [229]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[2178   19]
 [  44   10]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      2197
         1.0       0.34      0.19      0.24        54

    accuracy                           0.97      2251
   macro avg       0.66      0.59      0.61      2251
weighted avg       0.96      0.97      0.97      2251



/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


羅吉斯回歸

In [230]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [231]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4377   25]
 [ 161    5]]

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      4402
         1.0       0.17      0.03      0.05       166

    accuracy                           0.96      4568
   macro avg       0.57      0.51      0.52      4568
weighted avg       0.94      0.96      0.95      4568



In [232]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[2188    9]
 [  54    0]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.00      0.00      0.00        54

    accuracy                           0.97      2251
   macro avg       0.49      0.50      0.49      2251
weighted avg       0.95      0.97      0.96      2251



KNN

In [233]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [234]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4400    2]
 [ 149   17]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4402
         1.0       0.89      0.10      0.18       166

    accuracy                           0.97      4568
   macro avg       0.93      0.55      0.58      4568
weighted avg       0.96      0.97      0.95      4568



In [235]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[2187   10]
 [  52    2]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.17      0.04      0.06        54

    accuracy                           0.97      2251
   macro avg       0.57      0.52      0.52      2251
weighted avg       0.96      0.97      0.96      2251



SVM(因資料沒處理過所以會跑很慢，故不採用)

In [236]:
# from sklearn import svm 
# clf = svm.SVC(kernel = 'linear') 
# clf.fit(X_train,y_train) 

In [237]:
# #訓練
# from sklearn.metrics import classification_report,confusion_matrix
# clf_tpred= clf.predict(X_train)
# print(confusion_matrix(y_train,clf_tpred))
# print("======================================================\n")
# print(classification_report(y_train,clf_tpred))

In [238]:
# #測試
# clf_pred= clf.predict(X_test)
# print(confusion_matrix(y_test,clf_pred))
# print("======================================================\n")
# print(classification_report(y_test,clf_pred))

In [239]:
dataP33=dataMinMax[[' Liability-Assets Flag',' ROA(A) before interest and % after tax',' ROA(B) before interest and depreciation after tax',
       ' ROA(C) before interest and depreciation before interest',' Net worth/Assets', ' Debt ratio %',' Persistent EPS in the Last Four Seasons',
       ' Per Share Net profit before tax (Yuan 瞼)', ' Borrowing dependency',' Net Value Per Share (A)', ' Net Value Per Share (B)',
       ' Net Value Per Share (C)', ' Operating Profit Per Share (Yuan 瞼)',' Operating profit/Paid-in capital', ' Tax rate (A)',
       ' Operating Gross Margin', ' Realized Sales Gross Margin',' Cash Flow Per Share', ' Cash flow rate',' Contingent liabilities/Net worth',
       ' Net Value Growth Rate',' Cash Reinvestment %', ' Total Asset Growth Rate',' After-tax Net Profit Growth Rate',
       ' Regular Net Profit Growth Rate',' Quick Ratio', ' Research and development expense rate',' Net Income to Total Assets',
       ' Retained Earnings to Total Assets',' Net profit before tax/Paid-in capital',' Current Liability to Assets', ' Working Capital to Total Assets',
       " Net Income to Stockholder's Equity",' Current Liability to Current Assets', ' Liability to Equity',' Current Liability to Equity',
       ' Current Liabilities/Equity',' Working Capital/Equity', ' Total expense/Assets',' Equity to Long-term Liability', ' CFO to Assets',
       ' Cash/Total Assets',' Gross Profit to Sales',' Operating profit per person', ' Quick Assets/Total Assets',' Equity to Liability',
       ' Cash/Current Liability',' Operating Funds to Liability',' Inventory and accounts receivable/Net value',' Fixed Assets Turnover Frequency',
       ' Cash Flow to Total Assets',' Total Asset Turnover', ' Fixed Assets to Assets',' Cash Flow to Equity', ' Current Assets/Total Assets',
       ' Cash Flow to Liability', ' Revenue per person',' Total assets to GNP price', ' Quick Asset Turnover Rate']]

#PCA降維

In [240]:
#PCA降維
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
principalComponents = pca.fit_transform(dataP33)
dataPPCA33 = pd.DataFrame(data=principalComponents)
dataPPCA33.head()

,0,1,2,3,4,5
0,0.597,-0.005,0.164,-0.172,-0.251,-0.203
1,0.710,-0.002,0.057,-0.158,-0.158,-0.015
2,-0.068,0.258,-0.098,-0.244,0.073,-0.287
3,0.688,0.471,0.313,-0.116,0.503,-0.148
4,0.365,-0.061,0.062,-0.176,-0.153,-0.066


In [241]:
X=dataPPCA33[dataPPCA33.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33 ,random_state=333)

決策樹

In [242]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [243]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4401    1]
 [  39  127]]

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      4402
         1.0       0.99      0.77      0.86       166

    accuracy                           0.99      4568
   macro avg       0.99      0.88      0.93      4568
weighted avg       0.99      0.99      0.99      4568



In [244]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[2131   66]
 [  42   12]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      2197
         1.0       0.15      0.22      0.18        54

    accuracy                           0.95      2251
   macro avg       0.57      0.60      0.58      2251
weighted avg       0.96      0.95      0.96      2251



隨機森林

In [245]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [246]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4402    0]
 [  12  154]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4402
         1.0       1.00      0.93      0.96       166

    accuracy                           1.00      4568
   macro avg       1.00      0.96      0.98      4568
weighted avg       1.00      1.00      1.00      4568



In [247]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[2180   17]
 [  45    9]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      2197
         1.0       0.35      0.17      0.23        54

    accuracy                           0.97      2251
   macro avg       0.66      0.58      0.61      2251
weighted avg       0.96      0.97      0.97      2251



羅吉斯回歸

In [248]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [249]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4401    1]
 [ 150   16]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4402
         1.0       0.94      0.10      0.17       166

    accuracy                           0.97      4568
   macro avg       0.95      0.55      0.58      4568
weighted avg       0.97      0.97      0.95      4568



In [250]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[2188    9]
 [  49    5]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.36      0.09      0.15        54

    accuracy                           0.97      2251
   macro avg       0.67      0.54      0.57      2251
weighted avg       0.96      0.97      0.97      2251



KNN

In [251]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [252]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4384   18]
 [  98   68]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4402
         1.0       0.79      0.41      0.54       166

    accuracy                           0.97      4568
   macro avg       0.88      0.70      0.76      4568
weighted avg       0.97      0.97      0.97      4568



In [253]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[2177   20]
 [  47    7]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      2197
         1.0       0.26      0.13      0.17        54

    accuracy                           0.97      2251
   macro avg       0.62      0.56      0.58      2251
weighted avg       0.96      0.97      0.97      2251



SVM

In [254]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [255]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4402    0]
 [ 166    0]]

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      4402
         1.0       0.00      0.00      0.00       166

    accuracy                           0.96      4568
   macro avg       0.48      0.50      0.49      4568
weighted avg       0.93      0.96      0.95      4568



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [256]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[2197    0]
 [  54    0]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.00      0.00      0.00        54

    accuracy                           0.98      2251
   macro avg       0.49      0.50      0.49      2251
weighted avg       0.95      0.98      0.96      2251



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#LDA降維

In [257]:
#LDA降維
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1) #使用LDA降維，因此資料集的目標變數有2個類別，所以最多至1維
lda_x = lda.fit_transform(dataP33[dataP33.columns],dataMinMax['Bankrupt?'])
dataPLDA33 = pd.DataFrame(data=lda_x)
dataPLDA33.head()

,0
0,3.804
1,0.931
2,4.860
3,1.697
4,-0.061


In [258]:
X=dataPLDA33[dataPLDA33.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33 ,random_state=333)

決策樹

In [259]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [260]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4398    4]
 [  81   85]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4402
         1.0       0.96      0.51      0.67       166

    accuracy                           0.98      4568
   macro avg       0.97      0.76      0.83      4568
weighted avg       0.98      0.98      0.98      4568



In [261]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[2156   41]
 [  43   11]]

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      2197
         1.0       0.21      0.20      0.21        54

    accuracy                           0.96      2251
   macro avg       0.60      0.59      0.59      2251
weighted avg       0.96      0.96      0.96      2251



隨機森林

In [262]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [263]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4400    2]
 [  13  153]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4402
         1.0       0.99      0.92      0.95       166

    accuracy                           1.00      4568
   macro avg       0.99      0.96      0.98      4568
weighted avg       1.00      1.00      1.00      4568



In [264]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[2139   58]
 [  39   15]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      2197
         1.0       0.21      0.28      0.24        54

    accuracy                           0.96      2251
   macro avg       0.59      0.63      0.61      2251
weighted avg       0.96      0.96      0.96      2251



羅吉斯回歸

In [265]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [266]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4375   27]
 [ 123   43]]

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      4402
         1.0       0.61      0.26      0.36       166

    accuracy                           0.97      4568
   macro avg       0.79      0.63      0.67      4568
weighted avg       0.96      0.97      0.96      4568



In [267]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

[[2182   15]
 [  41   13]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      2197
         1.0       0.46      0.24      0.32        54

    accuracy                           0.98      2251
   macro avg       0.72      0.62      0.65      2251
weighted avg       0.97      0.98      0.97      2251



KNN

In [268]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [269]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4384   18]
 [  95   71]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4402
         1.0       0.80      0.43      0.56       166

    accuracy                           0.98      4568
   macro avg       0.89      0.71      0.77      4568
weighted avg       0.97      0.98      0.97      4568



In [270]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[2157   40]
 [  38   16]]

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      2197
         1.0       0.29      0.30      0.29        54

    accuracy                           0.97      2251
   macro avg       0.63      0.64      0.64      2251
weighted avg       0.97      0.97      0.97      2251



SVM

In [271]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [272]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4402    0]
 [ 154   12]]

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      4402
         1.0       1.00      0.07      0.13       166

    accuracy                           0.97      4568
   macro avg       0.98      0.54      0.56      4568
weighted avg       0.97      0.97      0.95      4568



In [273]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[2192    5]
 [  52    2]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.29      0.04      0.07        54

    accuracy                           0.97      2251
   macro avg       0.63      0.52      0.53      2251
weighted avg       0.96      0.97      0.97      2251



#ICA降維

In [274]:
#ICA降維
from sklearn.decomposition import FastICA
ica = FastICA(n_components=6)
Ica = ica.fit_transform(dataP33)
dataPICA33 = pd.DataFrame(data=Ica)
dataPICA33.head()

,0,1,2,3,4,5
0,0.007,-0.014,-0.022,0.007,0.014,0.005
1,0.011,-0.017,-0.007,0.005,0.015,0.002
2,-0.013,-0.001,-0.016,0.017,-0.004,-0.007
3,0.006,-0.016,-0.013,0.004,0.001,-0.040
4,0.003,-0.012,-0.009,0.004,0.008,0.005


In [275]:
X=dataPICA33[dataPICA33.columns] #降維後的欄位
y=dataMinMax['Bankrupt?'] #標準化後的Bankrupt?
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33 ,random_state=333)

決策樹

In [276]:
from sklearn import tree
DT = tree.DecisionTreeClassifier(max_depth=10)
DT= DT.fit(X_train, y_train)

In [277]:
#訓練
DT_pred= DT.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,DT_pred))
print("======================================================\n")
print(classification_report(y_train,DT_pred))

[[4399    3]
 [  32  134]]

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      4402
         1.0       0.98      0.81      0.88       166

    accuracy                           0.99      4568
   macro avg       0.99      0.90      0.94      4568
weighted avg       0.99      0.99      0.99      4568



In [278]:
#測試
DT_pred= DT.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,DT_pred))
print("======================================================\n")
print(classification_report(y_test,DT_pred))

[[2140   57]
 [  39   15]]

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98      2197
         1.0       0.21      0.28      0.24        54

    accuracy                           0.96      2251
   macro avg       0.60      0.63      0.61      2251
weighted avg       0.96      0.96      0.96      2251



隨機森林

In [279]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=15,max_features="sqrt",oob_score=True,random_state=3)
RF.fit(X_train.values,y_train.values)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:560: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_features='sqrt', n_estimators=15, oob_score=True,
                       random_state=3)

In [280]:
#訓練
RF_pred= RF.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,RF_pred))
print("======================================================\n")
print(classification_report(y_train,RF_pred))

[[4402    0]
 [  14  152]]

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4402
         1.0       1.00      0.92      0.96       166

    accuracy                           1.00      4568
   macro avg       1.00      0.96      0.98      4568
weighted avg       1.00      1.00      1.00      4568



In [281]:
#測試
RF_pred= RF.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,RF_pred))
print("======================================================\n")
print(classification_report(y_test,RF_pred))

[[2174   23]
 [  44   10]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      2197
         1.0       0.30      0.19      0.23        54

    accuracy                           0.97      2251
   macro avg       0.64      0.59      0.61      2251
weighted avg       0.96      0.97      0.97      2251



羅吉斯回歸

In [282]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [283]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_train)
print(confusion_matrix(y_train,LR_pred))
print("======================================================\n")
print(classification_report(y_train,LR_pred))

[[4402    0]
 [ 166    0]]

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      4402
         1.0       0.00      0.00      0.00       166

    accuracy                           0.96      4568
   macro avg       0.48      0.50      0.49      4568
weighted avg       0.93      0.96      0.95      4568



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [284]:
#測試
from sklearn.metrics import classification_report,confusion_matrix
LR_pred= LR.predict(X_test)
print(confusion_matrix(y_test,LR_pred))
print("======================================================\n")
print(classification_report(y_test,LR_pred))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[2197    0]
 [  54    0]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.00      0.00      0.00        54

    accuracy                           0.98      2251
   macro avg       0.49      0.50      0.49      2251
weighted avg       0.95      0.98      0.96      2251



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN

In [285]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) #K值(n_neighbors)預設為5
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [286]:
#訓練
knn_pred= knn.predict(X_train)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,knn_pred))
print("======================================================\n")
print(classification_report(y_train,knn_pred))

[[4385   17]
 [  90   76]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4402
         1.0       0.82      0.46      0.59       166

    accuracy                           0.98      4568
   macro avg       0.90      0.73      0.79      4568
weighted avg       0.97      0.98      0.97      4568



In [287]:
#測試
knn_pred= knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print("======================================================\n")
print(classification_report(y_test,knn_pred))

[[2173   24]
 [  46    8]]

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      2197
         1.0       0.25      0.15      0.19        54

    accuracy                           0.97      2251
   macro avg       0.61      0.57      0.59      2251
weighted avg       0.96      0.97      0.97      2251



SVM

In [288]:
from sklearn import svm 
clf = svm.SVC(kernel = 'linear') 
clf.fit(X_train,y_train) 

SVC(kernel='linear')

In [289]:
#訓練
from sklearn.metrics import classification_report,confusion_matrix
clf_tpred= clf.predict(X_train)
print(confusion_matrix(y_train,clf_tpred))
print("======================================================\n")
print(classification_report(y_train,clf_tpred))

[[4402    0]
 [ 166    0]]

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      4402
         1.0       0.00      0.00      0.00       166

    accuracy                           0.96      4568
   macro avg       0.48      0.50      0.49      4568
weighted avg       0.93      0.96      0.95      4568



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [290]:
#測試
clf_pred= clf.predict(X_test)
print(confusion_matrix(y_test,clf_pred))
print("======================================================\n")
print(classification_report(y_test,clf_pred))

[[2197    0]
 [  54    0]]

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2197
         1.0       0.00      0.00      0.00        54

    accuracy                           0.98      2251
   macro avg       0.49      0.50      0.49      2251
weighted avg       0.95      0.98      0.96      2251



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
